# **Dependencies**

In [7]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [9]:
!pip install rouge-score
!pip install rouge


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fd1acc95cac262c90c8c97db80484a2e7ac59360dd5bd13b72e44ce3ed5f1fb7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [19]:
!pip install tabulate

# **Telugu-English Sarcastic**

In [ ]:
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch

# Load your training data from Excel
train_excel_file_path = "te_en_sarcastic_train.csv"
train_df = pd.read_csv(train_excel_file_path)

# Define a custom dataset using the loaded DataFrame
train_dataset_sar_te = Dataset.from_pandas(train_df)

# Load the Pegasus-XSUM model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer_sar_te = PegasusTokenizer.from_pretrained(model_name)
model_sar_te = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer_sar_te(examples['text'], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    labels = tokenizer_sar_te(examples['title'], truncation=True, max_length=150, padding="max_length", return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset_sar_te = train_dataset_sar_te.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model_sar_te,
    args=training_args,
    train_dataset=tokenized_dataset_sar_te,
)

# Fine-tune the model
trainer.train()



model_save_path = "/content/drive/My Drive/NLP Project Models/sar_model_te"
model_sar_te.save_pretrained(model_save_path)

# Save the tokenizer to Google Drive
tokenizer_save_path = "/content/drive/My Drive/NLP Project Models/sar_tokenizer_te"
tokenizer_sar_te.save_pretrained(tokenizer_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/596 [00:00<?, ? examples/s]

Step,Training Loss
10,9.426500
20,8.805700
30,8.624800
40,8.624900
50,8.150900
60,8.124600
70,8.384500
80,8.014000
90,7.993200
100,8.021900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6, 'forced_eos_token_id': 1}


('/content/drive/My Drive/NLP Project Models/sar_tokenizer_te/tokenizer_config.json',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_te/special_tokens_map.json',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_te/spiece.model',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_te/added_tokens.json')

In [14]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset

# Load the fine-tuned model and tokenizer
sar_model_te = "/content/MyDrive/MyDrive/NLP Project Models/sar_model_te"  # Replace with the path to your fine-tuned model
sar_tokenizer_te = "/content/MyDrive/MyDrive/NLP Project Models/sar_tokenizer_te"  # Replace with the path to your fine-tuned tokenizer
fine_tuned_model_sar_te = PegasusForConditionalGeneration.from_pretrained(sar_model_te)
fine_tuned_tokenizer_sar_te = PegasusTokenizer.from_pretrained(sar_tokenizer_te)

# Load your test data from Excel
test_excel_file_path = "tel_eng_sarcastic_test.csv"
test_df = pd.read_csv(test_excel_file_path)

output_txt_file_path = "generated_headlines_sar_te.txt"
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for example in test_df["text"]:
        article_text = example

        # Tokenize the article text
        inputs = fine_tuned_tokenizer_sar_te(article_text, truncation=True, max_length=512,  return_tensors="pt")

        # Generate the headline
        output = fine_tuned_model_sar_te.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
        generated_headline = fine_tuned_tokenizer_sar_te.decode(output[0], skip_special_tokens=True)
        # Write the generated headline to the text file
        output_file.write(generated_headline + "\n")

print("generated_headline")



generated_headline


In [15]:
import pandas as pd

# Load your CSV file
csv_file_path = "tel_eng_sarcastic_test.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract the "title" column
titles = df["title"]

# Save the titles to a text file
output_txt_file_path = "actual_headlines_sar_te.txt"  # Replace with the path to your output text file
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for title in titles:
        output_file.write(title + "\n")


In [23]:
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import corpus_bleu
from tabulate import tabulate

def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences

translated_file = 'generated_headlines_sar_te.txt'
reference_file = 'actual_headlines_sar_te.txt'

# Read sentences from files
translated_sentences = read_sentences_from_file(translated_file)
reference_sentences = read_sentences_from_file(reference_file)


# Bleu Score
bleu_score = corpus_bleu([[refer] for refer in reference_sentences], translated_sentences)


#Rouge Score
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def calculate_ROUGE_score(ref_file, hyp_file):
    # Read reference and hypothesis text from files
    reference_text = read_text_file(ref_file)
    hypothesis_text = read_text_file(hyp_file)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
    # Calculate ROUGE scores
    scores = scorer.score(hypothesis_text, reference_text)
    return scores


rouge_scores = calculate_ROUGE_score(reference_file,translated_file)


# Extract precision, recall, and F-measure from ROUGE scores
rouge1_prec = rouge_scores['rouge1'].precision
rouge1_rec = rouge_scores['rouge1'].recall
rouge1_fmeas = rouge_scores['rouge1'].fmeasure

rouge2_prec = rouge_scores['rouge2'].precision
rouge2_rec = rouge_scores['rouge2'].recall
rouge2_fmeas = rouge_scores['rouge2'].fmeasure

rougeL_prec = rouge_scores['rougeL'].precision
rougeL_rec = rouge_scores['rougeL'].recall
rougeL_fmeas = rouge_scores['rougeL'].fmeasure

# Create a DataFrame to store BLEU and ROUGE scores along with precision, recall, and F-measure
data = {
    'Metric': ['BLEU Score', 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F-measure',
               'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F-measure',
               'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F-measure'],
    'Score': [bleu_score, rouge1_prec, rouge1_rec, rouge1_fmeas,
              rouge2_prec, rouge2_rec, rouge2_fmeas,
              rougeL_prec, rougeL_rec, rougeL_fmeas]
}
df_scores = pd.DataFrame(data)

# Display the scores using tabulate with fancy_grid style
table = tabulate(df_scores, headers='keys', tablefmt='fancy_grid', showindex=False, numalign="center", stralign="center", colalign=("center",))
print(table)

╒═══════════════════╤══════════╕
│      Metric       │  Score   │
╞═══════════════════╪══════════╡
│    BLEU Score     │ 0.401273 │
├───────────────────┼──────────┤
│ ROUGE-1 Precision │ 0.612939 │
├───────────────────┼──────────┤
│  ROUGE-1 Recall   │ 0.611915 │
├───────────────────┼──────────┤
│ ROUGE-1 F-measure │ 0.612427 │
├───────────────────┼──────────┤
│ ROUGE-2 Precision │ 0.188616 │
├───────────────────┼──────────┤
│  ROUGE-2 Recall   │ 0.188301 │
├───────────────────┼──────────┤
│ ROUGE-2 F-measure │ 0.188458 │
├───────────────────┼──────────┤
│ ROUGE-L Precision │ 0.295036 │
├───────────────────┼──────────┤
│  ROUGE-L Recall   │ 0.294543 │
├───────────────────┼──────────┤
│ ROUGE-L F-measure │ 0.29479  │
╘═══════════════════╧══════════╛


# **Telugu-English Non Sarcastic**

In [ ]:
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch

# Load your training data from Excel
train_excel_file_path = "tel_english_non_sarcastic_train.csv"
train_df = pd.read_csv(train_excel_file_path)

# Define a custom dataset using the loaded DataFrame
train_dataset_non_sar_gu = Dataset.from_pandas(train_df)

# Load the Pegasus-XSUM model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer_non_sar_te = PegasusTokenizer.from_pretrained(model_name)
model_non_sar_te = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer_non_sar_te(examples['text'], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    labels = tokenizer_non_sar_te(examples['title'], truncation=True, max_length=150, padding="max_length", return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset_non_sar_te = train_dataset_non_sar_gu.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model_non_sar_te,
    args=training_args,
    train_dataset=tokenized_dataset_non_sar_te,
)

# Fine-tune the model
trainer.train()


model_save_path = "/content/drive/My Drive/NLP Project Models/non_sar_model_te"
model_non_sar_te.save_pretrained(model_save_path)

# Save the tokenizer to Google Drive
tokenizer_save_path = "/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_te"
tokenizer_non_sar_te.save_pretrained(tokenizer_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Step,Training Loss
10,11.147300
20,10.461800
30,10.158900
40,9.880500
50,9.833000
60,9.718000
70,9.488800
80,9.324600
90,9.128400
100,9.010200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6, 'forced_eos_token_id': 1}


('/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_te/tokenizer_config.json',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_te/special_tokens_map.json',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_te/spiece.model',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_te/added_tokens.json')

In [17]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset

# Load the fine-tuned model and tokenizer
non_sar_model_te = "/content/MyDrive/MyDrive/NLP Project Models/non_sar_model_te"  # Replace with the path to your fine-tuned model
non_sar_tokenizer_te = "/content/MyDrive/MyDrive/NLP Project Models/non_sar_tokenizer_te"  # Replace with the path to your fine-tuned tokenizer
fine_tuned_model_non_sar_te = PegasusForConditionalGeneration.from_pretrained(non_sar_model_te)
fine_tuned_tokenizer_non_sar_te = PegasusTokenizer.from_pretrained(non_sar_tokenizer_te)

# Load your test data from Excel
test_excel_file_path = "tel_eng_non_sarcastic_test.csv"
test_df = pd.read_csv(test_excel_file_path)

output_txt_file_path = "generated_headlines_non_sar_te.txt"
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for example in test_df["text"]:
        article_text = example

        # Tokenize the article text
        inputs = fine_tuned_tokenizer_non_sar_te(article_text, truncation=True, max_length=512,  return_tensors="pt")

        # Generate the headline
        output = fine_tuned_model_non_sar_te.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
        generated_headline = fine_tuned_tokenizer_non_sar_te.decode(output[0], skip_special_tokens=True)
        # Write the generated headline to the text file
        output_file.write(generated_headline + "\n")

print("generated_headline")



generated_headline


In [34]:
import pandas as pd

# Load your CSV file
csv_file_path = "tel_eng_non_sarcastic_test.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract the "title" column
titles = df["title"]

# Save the titles to a text file
output_txt_file_path = "actual_headlines_non_sar_te.txt"  # Replace with the path to your output text file
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for title in titles:
        output_file.write(title + "\n")


In [35]:
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import corpus_bleu
from tabulate import tabulate

def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences

translated_file = 'generated_headlines_non_sar_te.txt'
reference_file = 'actual_headlines_non_sar_te.txt'

# Read sentences from files
translated_sentences = read_sentences_from_file(translated_file)
reference_sentences = read_sentences_from_file(reference_file)


# Bleu Score
bleu_score = corpus_bleu([[refer] for refer in reference_sentences], translated_sentences)


#Rouge Score
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def calculate_ROUGE_score(ref_file, hyp_file):
    # Read reference and hypothesis text from files
    reference_text = read_text_file(ref_file)
    hypothesis_text = read_text_file(hyp_file)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
    # Calculate ROUGE scores
    scores = scorer.score(hypothesis_text, reference_text)
    return scores



rouge1_prec = rouge_scores['rouge1'].precision
rouge1_rec = rouge_scores['rouge1'].recall
rouge1_fmeas = rouge_scores['rouge1'].fmeasure

rouge2_prec = rouge_scores['rouge2'].precision
rouge2_rec = rouge_scores['rouge2'].recall
rouge2_fmeas = rouge_scores['rouge2'].fmeasure

rougeL_prec = rouge_scores['rougeL'].precision
rougeL_rec = rouge_scores['rougeL'].recall
rougeL_fmeas = rouge_scores['rougeL'].fmeasure

# Create a DataFrame to store BLEU and ROUGE scores along with precision, recall, and F-measure
data = {
    'Metric': ['BLEU Score', 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F-measure',
               'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F-measure',
               'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F-measure'],
    'Score': [bleu_score, rouge1_prec, rouge1_rec, rouge1_fmeas,
              rouge2_prec, rouge2_rec, rouge2_fmeas,
              rougeL_prec, rougeL_rec, rougeL_fmeas]
}
df_scores = pd.DataFrame(data)

# Display the scores using tabulate with fancy_grid style
table = tabulate(df_scores, headers='keys', tablefmt='fancy_grid', showindex=False, numalign="center", stralign="center", colalign=("center",))
print(table)



╒═══════════════════╤══════════╕
│      Metric       │  Score   │
╞═══════════════════╪══════════╡
│    BLEU Score     │ 0.341774 │
├───────────────────┼──────────┤
│ ROUGE-1 Precision │ 0.871795 │
├───────────────────┼──────────┤
│  ROUGE-1 Recall   │    1     │
├───────────────────┼──────────┤
│ ROUGE-1 F-measure │ 0.931507 │
├───────────────────┼──────────┤
│ ROUGE-2 Precision │ 0.736842 │
├───────────────────┼──────────┤
│  ROUGE-2 Recall   │ 0.848485 │
├───────────────────┼──────────┤
│ ROUGE-2 F-measure │ 0.788732 │
├───────────────────┼──────────┤
│ ROUGE-L Precision │ 0.846154 │
├───────────────────┼──────────┤
│  ROUGE-L Recall   │ 0.970588 │
├───────────────────┼──────────┤
│ ROUGE-L F-measure │ 0.90411  │
╘═══════════════════╧══════════╛


# **Gujarati-English Sarcastic**

In [ ]:
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch

# Load your training data from Excel
train_excel_file_path = "gu_en_sarcastic_train.csv"
train_df = pd.read_csv(train_excel_file_path)

# Define a custom dataset using the loaded DataFrame
train_dataset_sar_gu = Dataset.from_pandas(train_df)

# Load the Pegasus-XSUM model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer_sar_gu = PegasusTokenizer.from_pretrained(model_name)
model_sar_gu = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer_sar_gu(examples['text'], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    labels = tokenizer_sar_gu(examples['title'], truncation=True, max_length=150, padding="max_length", return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset_sar_gu = train_dataset_sar_gu.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model_sar_gu,
    args=training_args,
    train_dataset=tokenized_dataset_sar_gu,
)

# Fine-tune the model
trainer.train()


model_save_path = "/content/drive/My Drive/NLP Project Models/sar_model_gu"
model_sar_gu.save_pretrained(model_save_path)

# Save the tokenizer to Google Drive
tokenizer_save_path = "/content/drive/My Drive/NLP Project Models/sar_tokenizer_gu"
tokenizer_sar_gu.save_pretrained(tokenizer_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/587 [00:00<?, ? examples/s]

Step,Training Loss
10,9.430600
20,8.788800
30,8.523200
40,8.479200
50,8.469200
60,8.207200
70,8.313600
80,7.964300
90,7.887500
100,7.949100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6, 'forced_eos_token_id': 1}


('/content/drive/My Drive/NLP Project Models/sar_tokenizer_gu/tokenizer_config.json',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_gu/special_tokens_map.json',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_gu/spiece.model',
 '/content/drive/My Drive/NLP Project Models/sar_tokenizer_gu/added_tokens.json')

In [ ]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset

# Load the fine-tuned model and tokenizer
non_sar_model_te = "/content/MyDrive/MyDrive/NLP Project Models/sar_model_gu"  # Replace with the path to your fine-tuned model
non_sar_tokenizer_te = "/content/MyDrive/MyDrive/NLP Project Models/sar_tokenizer_gu"  # Replace with the path to your fine-tuned tokenizer
fine_tuned_model_non_sar_te = PegasusForConditionalGeneration.from_pretrained(non_sar_model_te)
fine_tuned_tokenizer_non_sar_te = PegasusTokenizer.from_pretrained(non_sar_tokenizer_te)

# Load your test data from Excel
test_excel_file_path = "/content/guj_english_sarcastic_test.csv"
test_df = pd.read_csv(test_excel_file_path)

output_txt_file_path = "generated_headlines_sar_gu.txt"
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for example in test_df["text"]:
        article_text = example

        # Tokenize the article text
        inputs = fine_tuned_tokenizer_non_sar_te(article_text, truncation=True, max_length=512,  return_tensors="pt")

        # Generate the headline
        output = fine_tuned_model_non_sar_te.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
        generated_headline = fine_tuned_tokenizer_non_sar_te.decode(output[0], skip_special_tokens=True)
        # Write the generated headline to the text file
        output_file.write(generated_headline + "\n")

print("generated_headline")



In [32]:
import pandas as pd

# Load your CSV file
csv_file_path = "guj_english_sarcastic_test.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract the "title" column
titles = df["title"]

# Save the titles to a text file
output_txt_file_path = "actual_headlines_sar_gu.txt"  # Replace with the path to your output text file
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for title in titles:
        output_file.write(title + "\n")


In [37]:
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import corpus_bleu
from tabulate import tabulate

def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences

translated_file = '/content/generated_headlines_sar_gu.txt'
reference_file = 'actual_headlines_sar_gu.txt'

# Read sentences from files
translated_sentences = read_sentences_from_file(translated_file)
reference_sentences = read_sentences_from_file(reference_file)


# Bleu Score
bleu_score = corpus_bleu([[refer] for refer in reference_sentences], translated_sentences)


#Rouge Score
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def calculate_ROUGE_score(ref_file, hyp_file):
    # Read reference and hypothesis text from files
    reference_text = read_text_file(ref_file)
    hypothesis_text = read_text_file(hyp_file)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
    # Calculate ROUGE scores
    scores = scorer.score(hypothesis_text, reference_text)
    return scores



rouge1_prec = rouge_scores['rouge1'].precision
rouge1_rec = rouge_scores['rouge1'].recall
rouge1_fmeas = rouge_scores['rouge1'].fmeasure

rouge2_prec = rouge_scores['rouge2'].precision
rouge2_rec = rouge_scores['rouge2'].recall
rouge2_fmeas = rouge_scores['rouge2'].fmeasure

rougeL_prec = rouge_scores['rougeL'].precision
rougeL_rec = rouge_scores['rougeL'].recall
rougeL_fmeas = rouge_scores['rougeL'].fmeasure

# Create a DataFrame to store BLEU and ROUGE scores along with precision, recall, and F-measure
data = {
    'Metric': ['BLEU Score', 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F-measure',
               'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F-measure',
               'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F-measure'],
    'Score': [bleu_score, rouge1_prec, rouge1_rec, rouge1_fmeas,
              rouge2_prec, rouge2_rec, rouge2_fmeas,
              rougeL_prec, rougeL_rec, rougeL_fmeas]
}
df_scores = pd.DataFrame(data)

# Display the scores using tabulate with fancy_grid style
table = tabulate(df_scores, headers='keys', tablefmt='fancy_grid', showindex=False, numalign="center", stralign="center", colalign=("center",))
print(table)



╒═══════════════════╤══════════╕
│      Metric       │  Score   │
╞═══════════════════╪══════════╡
│    BLEU Score     │ 0.341774 │
├───────────────────┼──────────┤
│ ROUGE-1 Precision │ 0.871795 │
├───────────────────┼──────────┤
│  ROUGE-1 Recall   │    1     │
├───────────────────┼──────────┤
│ ROUGE-1 F-measure │ 0.931507 │
├───────────────────┼──────────┤
│ ROUGE-2 Precision │ 0.736842 │
├───────────────────┼──────────┤
│  ROUGE-2 Recall   │ 0.848485 │
├───────────────────┼──────────┤
│ ROUGE-2 F-measure │ 0.788732 │
├───────────────────┼──────────┤
│ ROUGE-L Precision │ 0.846154 │
├───────────────────┼──────────┤
│  ROUGE-L Recall   │ 0.970588 │
├───────────────────┼──────────┤
│ ROUGE-L F-measure │ 0.90411  │
╘═══════════════════╧══════════╛


# **Gujarati-English Non Sarcastic**

In [ ]:
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch

# Load your training data from Excel
train_excel_file_path = "guj_english_non_sarcastic_train.csv"
train_df = pd.read_csv(train_excel_file_path)

# Define a custom dataset using the loaded DataFrame
train_dataset_non_sar_gu = Dataset.from_pandas(train_df)

# Load the Pegasus-XSUM model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer_non_sar_gu = PegasusTokenizer.from_pretrained(model_name)
model_non_sar_gu = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer_non_sar_gu(examples['text'], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    labels = tokenizer_non_sar_gu(examples['title'], truncation=True, max_length=150, padding="max_length", return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset_non_sar_gu = train_dataset_non_sar_gu.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model_non_sar_gu,
    args=training_args,
    train_dataset=tokenized_dataset_non_sar_gu,
)

# Fine-tune the model
trainer.train()

model_save_path = "/content/drive/My Drive/NLP Project Models/non_sar_model_gu"
model_non_sar_gu.save_pretrained(model_save_path)

# Save the tokenizer to Google Drive
tokenizer_save_path = "/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_gu"
tokenizer_non_sar_gu.save_pretrained(tokenizer_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Step,Training Loss
10,11.103100
20,10.438000
30,10.204200
40,9.942200
50,9.831100
60,9.717600
70,9.468700
80,9.284300
90,9.186100
100,9.024200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6, 'forced_eos_token_id': 1}


('/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_gu/tokenizer_config.json',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_gu/special_tokens_map.json',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_gu/spiece.model',
 '/content/drive/My Drive/NLP Project Models/non_sar_tokenizer_gu/added_tokens.json')

In [30]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import Dataset

# Load the fine-tuned model and tokenizer
non_sar_model_te = "/content/MyDrive/MyDrive/NLP Project Models/non_sar_model_gu"  # Replace with the path to your fine-tuned model
non_sar_tokenizer_te = "/content/MyDrive/MyDrive/NLP Project Models/non_sar_tokenizer_gu"  # Replace with the path to your fine-tuned tokenizer
fine_tuned_model_non_sar_te = PegasusForConditionalGeneration.from_pretrained(non_sar_model_te)
fine_tuned_tokenizer_non_sar_te = PegasusTokenizer.from_pretrained(non_sar_tokenizer_te)

# Load your test data from Excel
test_excel_file_path = "/content/guj_english_non_sarcastic_test.csv"
test_df = pd.read_csv(test_excel_file_path)

output_txt_file_path = "generated_headlines_non_sar_gu.txt"
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for example in test_df["text"]:
        article_text = example

        # Tokenize the article text
        inputs = fine_tuned_tokenizer_non_sar_te(article_text, truncation=True, max_length=512,  return_tensors="pt")

        # Generate the headline
        output = fine_tuned_model_non_sar_te.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
        generated_headline = fine_tuned_tokenizer_non_sar_te.decode(output[0], skip_special_tokens=True)
        # Write the generated headline to the text file
        output_file.write(generated_headline + "\n")

print("generated_headline")



KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Load your CSV file
csv_file_path = "guj_english_non_sarcastic_test.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract the "title" column
titles = df["title"]

# Save the titles to a text file
output_txt_file_path = "actual_headlines_non_sar_gu.txt"  # Replace with the path to your output text file
with open(output_txt_file_path, "w", encoding="utf-8") as output_file:
    for title in titles:
        output_file.write(title + "\n")


In [41]:
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import corpus_bleu
from tabulate import tabulate

def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences

translated_file = '/content/generated_headlines_non_sar_gu.txt'
reference_file = 'actual_headlines_non_sar_gu.txt'

# Read sentences from files
translated_sentences = read_sentences_from_file(translated_file)
reference_sentences = read_sentences_from_file(reference_file)


# Bleu Score
bleu_score = corpus_bleu([[refer] for refer in reference_sentences], translated_sentences)


#Rouge Score
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def calculate_ROUGE_score(ref_file, hyp_file):
    # Read reference and hypothesis text from files
    reference_text = read_text_file(ref_file)
    hypothesis_text = read_text_file(hyp_file)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
    # Calculate ROUGE scores
    scores = scorer.score(hypothesis_text, reference_text)
    return scores



rouge1_prec = rouge_scores['rouge1'].precision
rouge1_rec = rouge_scores['rouge1'].recall
rouge1_fmeas = rouge_scores['rouge1'].fmeasure

rouge2_prec = rouge_scores['rouge2'].precision
rouge2_rec = rouge_scores['rouge2'].recall
rouge2_fmeas = rouge_scores['rouge2'].fmeasure

rougeL_prec = rouge_scores['rougeL'].precision
rougeL_rec = rouge_scores['rougeL'].recall
rougeL_fmeas = rouge_scores['rougeL'].fmeasure

# Create a DataFrame to store BLEU and ROUGE scores along with precision, recall, and F-measure
data = {
    'Metric': ['BLEU Score', 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F-measure',
               'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F-measure',
               'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F-measure'],
    'Score': [bleu_score, rouge1_prec, rouge1_rec, rouge1_fmeas,
              rouge2_prec, rouge2_rec, rouge2_fmeas,
              rougeL_prec, rougeL_rec, rougeL_fmeas]
}
df_scores = pd.DataFrame(data)

# Display the scores using tabulate with fancy_grid style
table = tabulate(df_scores, headers='keys', tablefmt='fancy_grid', showindex=False, numalign="center", stralign="center", colalign=("center",))
print(table)



╒═══════════════════╤══════════╕
│      Metric       │  Score   │
╞═══════════════════╪══════════╡
│    BLEU Score     │ 0.341774 │
├───────────────────┼──────────┤
│ ROUGE-1 Precision │ 0.871795 │
├───────────────────┼──────────┤
│  ROUGE-1 Recall   │    1     │
├───────────────────┼──────────┤
│ ROUGE-1 F-measure │ 0.931507 │
├───────────────────┼──────────┤
│ ROUGE-2 Precision │ 0.736842 │
├───────────────────┼──────────┤
│  ROUGE-2 Recall   │ 0.848485 │
├───────────────────┼──────────┤
│ ROUGE-2 F-measure │ 0.788732 │
├───────────────────┼──────────┤
│ ROUGE-L Precision │ 0.846154 │
├───────────────────┼──────────┤
│  ROUGE-L Recall   │ 0.970588 │
├───────────────────┼──────────┤
│ ROUGE-L F-measure │ 0.90411  │
╘═══════════════════╧══════════╛
